# Import and Def

In [1]:
test = False

# Create the output dictionary
dout = {}

In [2]:
import os, pickle
from math import sqrt
import numpy as np

In [3]:
def sumWithUnc(nList, scale=1.):
    x = 0
    dx = 0
    for b in nList:
        x += b[0]
        dx += b[1]**2
    dx = sqrt(dx)
    
    x *= scale
    dx *= scale
    print '{:1.3e} +/- {:1.3e} ({:.2f}%)'.format(x, dx, 100*np.abs(dx)/x)
    return np.array([x, dx])

In [4]:
def multWithUnc(nList):
    x = 1
    auxD = 0
    for b, db in nList:
        x *= b
        auxD += (db/b)**2
    dx = x*np.sqrt(auxD)
    return np.array([x, dx])

In [5]:
def divideWithUnc(n, d):
    x = n[0]/d[0]
    dx = x*np.hypot(n[1]/n[0], d[1]/d[0])
    return np.array([x, dx])

In [6]:
def computeBr(dl):
    r = 1
    e2_dr_r = 0
    for c in dl:
        if isinstance(c, float):
            r *= c
        else:
            r *= c[0]
            e2_dr_r += (c[1]/c[0])**2
    dr = sqrt(e2_dr_r)*r
    print '{:1.3e} +/- {:1.3e} ({:.2f}%)'.format(r, dr, 100*dr/r)
    return [r, dr]

Scratches

In [7]:
divideWithUnc([0.42, 0.12], [9.55, 0.69])

array([0.04397906, 0.01296099])

# Main signal

In [8]:
# https://pdglive.lbl.gov/BranchingRatio.action?desig=1&parCode=S035
tau_to_MuNuNu = [0.1739, 0.0004]

# https://pdglive.lbl.gov/Particle.action?init=0&node=S032&home=MXXX035#decayclump_C (Gamma 35)
Du_to_piK = [0.03946, 0.00030] # D0 -> K- pi+, anti-D0 -> K+ pi-

# https://pdglive.lbl.gov/Particle.action?init=0&node=M062&home=MXXX035 (Gamma 1)
Dst_to_piDu = [0.677, 0.005]

if test:
    Du_to_piK = [1., 0.]
    Dst_to_piDu = [1., 0]

# https://pdglive.lbl.gov/Particle.action?init=0&node=S042&home=MXXX045 (Gamma 6)
Bd_to_DstMuNu = np.array([5.06e-2, 0.12e-2])

In [9]:
dout['mu'] = computeBr([Bd_to_DstMuNu, Dst_to_piDu[0], Du_to_piK[0]])

# Tau gets everything but R(D*)
dout['tau'] = computeBr([dout['mu'], tau_to_MuNuNu[0]])

1.352e-03 +/- 3.206e-05 (2.37%)
2.351e-04 +/- 5.575e-06 (2.37%)


# $D^{**}$ background

In [10]:
# https://pdglive.lbl.gov/Particle.action?init=0&node=S041&home=MXXX045#decayclump_A (Gamma 12)
Bu_to_DstPiMuNu = np.array([6e-3, 0.4e-3])
dout['Bu_MuDstPi'] = computeBr([Bu_to_DstPiMuNu, Dst_to_piDu[0], Du_to_piK[0]])

# Assuming isospin and compativle with Gamma(B0) 12/2
Bd_to_DstPiMuNu = 0.5*Bu_to_DstPiMuNu
dout['Bd_MuDstPi'] = computeBr([Bd_to_DstPiMuNu, Dst_to_piDu[0], Du_to_piK[0]])

1.603e-04 +/- 1.069e-05 (6.67%)
8.014e-05 +/- 5.343e-06 (6.67%)


In [11]:
# https://inspirehep.net/literature/1385752
RstPipPim = np.array([0.019, np.hypot(0.005, 0.004)])

Bd_to_DstPipPimMuNu = multWithUnc([Bd_to_DstMuNu, RstPipPim])
Bd_to_DstPi0Pi0MuNu = Bd_to_DstPipPimMuNu/4 # isospin symm
dout['Bd_MuDstPiPi'] = computeBr([Bd_to_DstPipPimMuNu*5./4., Dst_to_piDu[0], Du_to_piK[0]])

Bu_to_DstPiPiMuNu = Bd_to_DstPipPimMuNu/2 # isospin symm
dout['Bu_MuDstPiPi'] = computeBr([Bu_to_DstPiPiMuNu, Dst_to_piDu[0], Du_to_piK[0]])

3.210e-05 +/- 1.085e-05 (33.78%)
1.284e-05 +/- 4.338e-06 (33.78%)


In [12]:
RDstst = 0.2

dout['Bu_TauDstPi'] = computeBr([dout['Bu_MuDstPi'], RDstst, tau_to_MuNuNu])
dout['Bd_TauDstPi'] = computeBr([dout['Bd_MuDstPi'], RDstst, tau_to_MuNuNu])
dout['Bd_TauDstPiPi'] = computeBr([dout['Bd_MuDstPiPi'], RDstst, tau_to_MuNuNu])
dout['Bu_TauDstPiPi'] = computeBr([dout['Bu_MuDstPiPi'], RDstst, tau_to_MuNuNu])

5.575e-06 +/- 3.719e-07 (6.67%)
2.787e-06 +/- 1.859e-07 (6.67%)
1.117e-06 +/- 3.772e-07 (33.78%)
4.466e-07 +/- 1.509e-07 (33.78%)


In [13]:
Bs_to_DstKMuNu = np.array([5.9e-3, 1.5e-3])
dout['Bs_MuDstK'] = computeBr([Bs_to_DstKMuNu, Dst_to_piDu[0], Du_to_piK[0]])

dout['Bs_TauDstK'] = computeBr([dout['Bs_MuDstK'], RDstst, tau_to_MuNuNu])

1.576e-04 +/- 4.007e-05 (25.42%)
5.482e-06 +/- 1.394e-06 (25.42%)


# Double charm background

## $D_{(s)} \to \mu X$

In [14]:
Du_to_MuX_list = [ # [10^-3]
    [34.1, 0.4],
    [18.9, 2.4],
    [2.67, 0.12],
    [1.50, 0.12],
    [0.76, 0.30],
    [0.77, 0.16],
    [0.39, 0.01],
    [0.30, 0.30],
    [1.45, 0.07]
]

print 'Total processes:', len(Du_to_MuX_list)

Du_to_MuX = sumWithUnc(Du_to_MuX_list, scale=1e-3)

Total processes: 9
6.084e-02 +/- 2.482e-03 (4.08%)


In [15]:
Dd_to_MuX_list = [ # [10^-3]
    [87.6, 1.9],
    [52.7, 1.5],
    [3.5, 0.15],
    [2.77, 0.40],
    [1.0, 1.0],
    [2.4, 0.4],
    [1.69, 0.11],
    [1.11, 0.07],
    [0.20, 0.04],
    [2.45, 0.10],
    [1.9, 0.5],
    [0.95, 0.01],
    [0.37, 0.02],
    [0.20, 0.05]
]

print 'Total processes:', len(Dd_to_MuX_list)

Dd_to_MuX = sumWithUnc(Dd_to_MuX_list, scale=1e-3)

Total processes: 14
1.588e-01 +/- 2.736e-03 (1.72%)


In [16]:
Ds_to_MuX_list = [ # [10^-3]
    [23.9, 1.6],
    [23.2, 0.8],
    [8.0, 0.7],
    [3.4, 0.4],
    [2.15, 0.28],
    [9.31, 0.39],
    [5.49, 0.39]
]

print 'Total processes:', len(Ds_to_MuX_list)

Ds_to_MuX = sumWithUnc(Ds_to_MuX_list, scale=1e-3)

Total processes: 7
7.545e-02 +/- 2.057e-03 (2.73%)


## $B^0 \to D^*D_{(s)}X$

In [17]:
Bd_to_DstDuX_list = [ # [10^-3]
    [2.47, 0.21],
    [1.24, 0.0],
    [10.6, 0.9],
    [5.3, 0.0],
    [2*5.43, 2*0.47],
    [2*2.7, 0.0],
    [2*0.54, 2*0.04],
]
print 'Total processes:', len(Bd_to_DstDuX_list)+3
print 'Sum of Br'
Bd_to_DstDuX = sumWithUnc(Bd_to_DstDuX_list, scale=1e-3)

print 'Sample norm'
dout['Bd_DstDu'] = computeBr([Bd_to_DstDuX, Du_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Total processes: 10
Sum of Br
3.695e-02 +/- 1.321e-03 (3.57%)
Sample norm
6.006e-05 +/- 3.257e-06 (5.42%)


In [18]:
Bd_to_DstDdX_list = [ # [10^-3]
    [2*3.2, 2*0.25],
    [2*1.6, 0.],
    [2*2.67, 2*0.23],
    [2*1.33, 0.],
    [2*0.26, 2*0.02],
    [2*0.61, 2*0.15]
]
print 'Total processes:', len(Bd_to_DstDdX_list)+6
print 'Sum of Br'
Bd_to_DstDdX = sumWithUnc(Bd_to_DstDdX_list, scale=1e-3)

print 'Sample norm'
dout['Bd_DstDd'] = computeBr([Bd_to_DstDdX, Dd_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Total processes: 12
Sum of Br
1.934e-02 +/- 7.438e-04 (3.85%)
Sample norm
8.207e-05 +/- 3.458e-06 (4.21%)


In [19]:
Bd_to_DstDsX_list = [ # [10^-3]
    [8.0, 1.1],
    [17.7, 0.14],
    [1.5, 1.0]
]
print 'Total processes:', len(Bd_to_DstDsX_list)
print 'Sum of Br'
Bd_to_DstDsX = sumWithUnc(Bd_to_DstDsX_list, scale=1e-3)

print 'Sample norm'
dout['Bd_DstDs'] = computeBr([Bd_to_DstDsX, Ds_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Total processes: 3
Sum of Br
2.720e-02 +/- 1.493e-03 (5.49%)
Sample norm
5.482e-05 +/- 3.360e-06 (6.13%)


## $B^+ \to D^*DX$

In [20]:
Bu_to_DstDuX = [23.3e-3, 2e-3]

print 'Sample norm'
dout['Bu_DstDu'] = computeBr([Bu_to_DstDuX, Du_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
3.787e-05 +/- 3.599e-06 (9.50%)


In [21]:
Bu_to_DstDdX = [2.11e-3, 0.2e-3]

print 'Sample norm'
dout['Bu_DstDd'] = computeBr([Bu_to_DstDdX, Dd_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
8.953e-06 +/- 8.626e-07 (9.63%)


## $ B \to DD_{s1}$

In [22]:
Bd_to_DDs1X = [0.785e-3, 0.3e-3] # Hc -> muX already factored in

print 'Sample norm'
dout['Bd_DDs1'] = computeBr([Bd_to_DDs1X, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
2.097e-05 +/- 8.014e-06 (38.22%)


In [23]:
Bu_to_DDs1X = [1.5e-3, 0.5e-3]
Ds1_to_DstK0 = [0.5, 0.1]

print 'Sample norm'
dout['Bu_DDs1'] = computeBr([Bu_to_DDs1X, Ds1_to_DstK0, Du_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
1.219e-06 +/- 4.765e-07 (39.09%)


## $B_s \to D^*D_sX$

In [24]:
Bs_to_DstDsX = [30.9e-3, 0.]

print 'Sample norm'
dout['Bs_DstDs'] = computeBr([Bs_to_DstDsX, Ds_to_MuX, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
6.228e-05 +/- 1.698e-06 (2.73%)


## Others

In [25]:
B_to_DstDXX = [5e-3, 0.3e-3] # Random norm

print 'Sample norm'
dout['B_DstDXX'] = computeBr([B_to_DstDXX, Dst_to_piDu[0], Du_to_piK[0]])

Sample norm
1.336e-04 +/- 8.014e-06 (6.00%)


# Ancillary

In [26]:
# https://pdglive.lbl.gov/Particle.action?init=0&node=M018&home=MXXX020
Kst_to_KPi = [1., 0.]

# https://pdglive.lbl.gov/Particle.action?init=0&node=M070&home=MXXX025 (Gamma 7)
JPsi_to_MuMu = [5.961e-2, 0.033e-2]

# https://pdglive.lbl.gov/BranchingRatio.action?desig=22&parCode=S042&home=MXXX045
Bd_to_JPsiKst = [1.27e-3, 0.05e-3]
dout['JPsiKst'] = computeBr([Bd_to_JPsiKst, JPsi_to_MuMu, Kst_to_KPi])

# https://pdglive.lbl.gov/BranchingRatio.action?desig=3&parCode=S041
Bu_to_JPsiK = [1.00e-3, 0.05e-3]
dout['JPsiK'] = computeBr([Bu_to_JPsiK, JPsi_to_MuMu, Kst_to_KPi])

7.570e-05 +/- 3.010e-06 (3.98%)
5.961e-05 +/- 2.999e-06 (5.03%)


# Dump the output dictionary

In [27]:
# fileName = '../data/forcedDecayChannelsFactors_v2.pickle'
fileName = '/storage/af/group/rdst_analysis/BPhysics/data/forcedDecayChannelsFactors_v2.pickle'
if test:
    fileName = fileName.replace('v2', 'test')
pickle.dump(dout, open(fileName, 'wb'))